In [ ]:
"""
columns_to_keep = df.filter(['name', 'firstName', 'lastName', 'slug', 'shortName', 'position',
       'jerseyNumber', 'height', 'preferredFoot', 'userCount', 'id',
       'shirtNumber', 'dateOfBirthTimestamp', 'contractUntilTimestamp','team.id',
       'proposedMarketValue','country.alpha2', 'country.name',
       'proposedMarketValueRaw.value', 'proposedMarketValueRaw.currency']
       ).columns

columns_to_keep

df_filtered = df[columns_to_keep]
df_filtered

"""